In [ ]:
# Imports cell
import mercury as mr

from random import randint, seed
from dotenv import load_dotenv
import sys, pickle, shutil, os.path
from tud_sumo.simulation import Simulation
from tud_sumo.plot import Plotter, MultiPlotter
import pandas as pd
from lxml import etree


In [ ]:
app = mr.App(title="CIEM6220-RTS-Integration project",description="Interactive Traffic Management / AV simulation.",allow_download=False,allow_share=False)

# CIEM6220 - Road Traffic Systems - Unit 1 2024/25

## Using this interactive notebook
**Please refer to the course's Brightspace page. Find more information on this notebook in the README.md and INSTALL.md files.**

- Choose your input parameters on the controls pane (<-)
- Traffic management needs to be activated in the corresponding menu
- Changes in behavioural model parameters must be confirmed by clicking on the *Update vehicle behaviour parameters* button. 
- Click on (re-)Run simulation to let the notebook interact with the simulation backend (SUMO)
- Results are visualised below, under "Simulation results"
- If "Use cache" is active, available simulation results will be loaded quickly, where feasible, instead of re-running the full simulation. *These results will not reflect changes in Behavioural Adaptation*.
    - This package is delivered with an empty cache folder. Cache needs to be generated locally, by executing **populate_cache.py**.
- If "Compare with previous simulation" is active when selecting "(re)-Run simulation", the latest simulation results will be kept and comparative KPIs will be shown for both that and the newly adapted inputs.
- If "Store simulation results" is selected, simulation results willbe stored in the *./experiments* folder. Simulation results are stored in pickled Pandas dataframes. Results can therefore be inspected manually through Pandas, other than through this notebook.
- If "Show simulation" is selected, SUMO's graphical user interface will open when a simulation is executed.

In [ ]:
#Graphical user interface

run_sim_button = mr.Button(label='(re-)Run simulation')
penRate_slider = mr.Slider(value=0,min=0,max=1,label="AV Penetration Rate [%]",step=0.1)
show_Control_toggle = mr.Checkbox(value=False,label="Show Traffic Management options")

if show_Control_toggle.value:
    VSL_toggle = mr.Checkbox(value=False,label="Activate Variable Speed Limit control") #Later: adjustable thresholds toggle like above
    VSL_threshold_70_input = mr.Numeric(value=80,min=20,max=130, label="vMax=70km/h, if weaving speed lower than..",step=10)
    VSL_threshold_50_input = mr.Numeric(value=60,min=20,max=130, label="vMax=50km/h, if weaving speed lower than..",step=10)

    RM_toggle = mr.Checkbox(value=False,label="Activate Ramp Metering control") #Later: adjustable thresholds toggle like above
    RM_threshold_input = mr.Numeric(value=130,min=20,max=130,label="RM Active, if weaving speed lower than..",step=10)

show_BA_toggle = mr.Checkbox(value=False,label="Show Behavioral Adaptation options")
if show_BA_toggle.value:
    #Sliders that define what % of either population was updated - vTypes need to be reassigned accordingly
    HV_BA_penrate_slider = mr.Slider(value=1,min=0,max=1,label="Percentage of Human-Driven Vehicles involved in BA:")
    CACC_BA_penrate_slider = mr.Slider(value=1,min=0,max=1,label="Percentage of CACC Vehicles involved in BA:")

    #Human Vehicles
    HV_minGap_slider = mr.Slider(value=0.25,min=0.05,max=1.25,label="Human-Driven Vehicle Minimum Desired Gap [m]",step=0.05)
    HV_accel_slider = mr.Slider(value=1.5,min=0.05,max=4,label="Human-Driven Vehicle Accelleration [m/s^2]",step=0.1)
    HV_decel_slider = mr.Slider(value=2,min=0.05,max=4,label="Human-Driven Vehicle Deceleration [m/s^2]",step=0.05)
    HV_sigma_slider = mr.Slider(value=0.5,min=0,max=1,label="Human-Driven Vehicle Driver imperfection [0=perfect,1=imperfect]",step=0.05)
    HV_tau_slider = mr.Slider(value=1,min=0.05,max=5,label="Human-Driven Vehicle Desired Minimum Time Headway [s]",step=0.05)
    #Autonomous Vehicles
    CACC_minGap_slider = mr.Slider(value=0.25,min=0.05,max=1.25,label="CACC Vehicle Minimum Desired Gap [m]",step=0.05)
    CACC_accel_slider = mr.Slider(value=1.5,min=0.05,max=4,label="CACC Vehicle Accelleration [m/s^2]",step=0.1)
    CACC_decel_slider = mr.Slider(value=2,min=0.05,max=4,label="CACC Vehicle Deceleration [m/s^2]",step=0.05)
    CACC_sigma_slider = mr.Slider(value=0.5,min=0,max=1,label="CACC Vehicle Driver imperfection [0=perfect,1=imperfect]",step=0.05)
    CACC_tau_slider = mr.Slider(value=1,min=0.05,max=5,label="CACC Vehicle Desired Minimum Time Headway [s]",step=0.05)

    CACC_SCG = mr.Slider(value=-0.4,min=-0.9,max=0,label="Speed control: rate of positioning deviation",step=0.05)
    CACC_GCGG = mr.Slider(value=0.005,min=0,max=0.05,label="Gap closing: rate of the positioning deviation ",step=0.005)
    CACC_GCGGD = mr.Slider(value=0.05,min=0,max=0.5,label="Gap closing: rate of positioning deviation derivative",step=0.05)
    CACC_GCG = mr.Slider(value=0.45,min=0,max=1,label="Gap control: rate of the positioning deviation ",step=0.05)
    CACC_GCGD = mr.Slider(value=0.0125,min=0,max=0.5,label="Gap control: rate of positioning deviation derivative",step=0.0125)
    CACC_CAG = mr.Slider(value=0.45,min=0,max=1,label="Collision avoidance: rate of the positioning derivative",step=0.05)
    CACC_CAGD = mr.Slider(value=0.05,min=0,max=0.5,label="Collision avoidance: rate of the positioning deviation derivative",step=0.05)  
    

    #Update button
    BA_button = mr.Button(label='Update vehicle behaviour parameters.')

compare_sim_results_toggle = mr.Checkbox(value=False,label="Compare with previous simulation")
use_cache_toggle = mr.Checkbox(value=False,label="Use cache")
if not use_cache_toggle.value:
    save_sim_toggle = mr.Checkbox(value=False,label="Store simulation results")
    see_GUI_toggle = mr.Checkbox(value=False,label="Show simulation")

else:
    #compare_sim_results_toggle = mr.Checkbox(value=False,label="Compare with previous simulation",disabled=True)
    save_sim_toggle = mr.Checkbox(value=False,label="Store simulation results",disabled=True)
    see_GUI_toggle = mr.Checkbox(value=False,label="Show simulation",disabled=True)

    



In [ ]:
# Convenience functions
import re
def readSimDescription(filename=""):
    description=""
    if filename != "" and filename.split('.')[-1]=='txt' and os.path.exists(filename): #some nice checking that this is a text file with desired outputs
        raw_data=""
        with open(filename,'r') as f:
            raw_data = f.read().replace('\n', '')
        try: #this bit is try catch'd, if it fails description remains = ""
            raw_data=raw_data.split("Description:")[1]
            raw_data=raw_data.split("*")[0]
            raw_data=re.sub('\s{2,}', ' ', raw_data)    
            raw_data=raw_data.replace('|',' ')
            description=raw_data
        except Exception:
            pass
                

    return description

In [ ]:
#Simulation backend
simComplete=False
savesDir="./experiments/"
cacheDir="./cache/"
old_sim = None

def updateVehicleClasses(HV_user_values_dict=dict(),CACC_user_values_dict=dict(),rouFile='./a20_exercise/a20_scenario/a20.rou.xml'): #collects user input for Behavioural Adaptation and updates the corresponding classes in the scenario's rou file
    CACC_default_values_dict={'minGap':0.25,'accel':1.5,'decel':2,'tau':1,'speedControlGainCACC':-0.4,'gapClosingControlGainGap':0.005,'gapClosingControlGainGapDot':0.05, 
                          'gapControlGainGap':0.45,'gapControlGainGapDot':0.0125,'collisionAvoidanceGainGap':0.45,'collisionAvoidanceGainGapDot':0.05}
    HV_default_values_dict={'minGap':0.25,'accel':1.5,'decel':2,'sigma':0.5,'tau':1}
    tree=etree.parse(rouFile)
    root=tree.getroot()
    for element in root.iter("vType"):
        attribs=element.attrib
        if element.get("id")=="BA_AV":
            #Adapt parameters based on user input
            for k in CACC_default_values_dict.keys():
                if k in CACC_user_values_dict:
                    attribs[k]=str(CACC_user_values_dict[k])
                else:
                    attribs[k]=str(CACC_default_values_dict[k])
            

        if element.get("id")=="BA_cars":
            #Adapt parameters based on user input
            for k in HV_default_values_dict.keys():
                if k in HV_user_values_dict:
                    attribs[k]=str(HV_user_values_dict[k])
                else:
                    attribs[k]=str(HV_default_values_dict[k])

    tree.write(rouFile)


def runScenario(my_sim,useVSL=False,useRM=False):
    #Define some convenience dictionary to store vehicle ids and whether they are AVs or not
    vehicleClass={}
    # Start the simulation, defining the sumo config files. Add "-gui" to the command line to run with the GUI.
    my_sim.start("a20_exercise/a20_scenario/a20.sumocfg", get_individual_vehicle_data=False, gui=("-gui" in sys.argv) or forceGUI,
                 seed=sim_seed, units="metric", suppress_pbar=True) # Units can either be metric (km,kmph)/imperial (mi,mph)/UK (km,mph). All data collected is in these units.
    
    # Add demand from a '.csv' file.
    # my_sim.load_demand("a20_scenario/demand.csv")

    #Add a tracked junction to the intersection with ID "utsc", which will track signal phases/times.
    my_sim.add_tracked_junctions({"utsc": {"flow_params": {"inflow_detectors": ["utsc_n_in_1", "utsc_n_in_2", "utsc_w_in", "utsc_e_in"],
                                                       "outflow_detectors": ["utsc_w_out", "utsc_e_out"],
                                                       "vehicle_types": ["cars", "lorries", "motorcycles", "vans"]}}})

    # Set traffic signal phases. The junc_phases dict can be used for multiple junctions.
    # set_m_phases() can be used to set phases according to different movements (here, movements 1 & 2)
    phases = {"phases": {1: ["G", "y", "r"], 2: ["r", "G", "y"]},
             "times":  {1: [ 27,  3,   20], 2: [ 30,  17,  3]},
             "masks":  {1: "1100",          2: "0011"}}
    
    my_sim.set_m_phases({"utsc": phases})

    # This is equivalent to: my_sim.set_phases({"utsc": {"phases": ["GGrr", "yyrr", "rrGG", "rryy"], "times": [27, 3, 17, 3]}})

    # Add a ramp meter to the junction with ID "crooswijk_meter". The junc_params dict can be used to -> doesn't 'add' anything. the TL is already there and has a certain activation, hardcoded in the network
    # define meter specifc parameters (min/max rate, spillback tracking or queue detectors) and flow specific
    # parameters (inflow/outflow detectors used to calculate in/out flow).
    my_sim.add_tracked_junctions({"crooswijk_meter": {'meter_params': {'min_rate': 200, 'max_rate': 4000, 'queue_detector': "cw_ramp_queue"},
                                                    'flow_params': {'inflow_detectors': ["cw_ramp_inflow", "cw_rm_upstream"], 'outflow_detectors': ["cw_rm_downstream"]}},
                                "a13_meter": {'meter_params': {'min_rate': 200, 'max_rate': 4000, 'queue_detector': "a13_ramp_queue"},
                                                'flow_params': {'inflow_detectors': ["a13_ramp_inflow", "a13_rm_upstream"], 'outflow_detectors': ["a13_rm_downstream"]}}})
    
    my_sim.add_controllers({"vsl1": {"type": "VSL", "geometry_ids": ["629633083", "629633083.833","61121496", "61121498", "54374946", "126730044", "126729958", "126710337"]}})

    my_sim.add_controllers({"vsl2": {"type": "VSL", "geometry_ids": ["1191885780", "1191885781","1191885778", "126730088", "491000664", "29324581", "1191885774"]}})


    # Add tracked edges. This will track some basic information, such as average speed etc, but can also
    # be used to create space-time diagrams as individual vehicle speeds and positions are tracked.
    tracked_edges = ["629633083", "629633083.833", "61121496", "61121498", "54374946" , "126730044" , "126729958" , "126710337" , "1191885785", "699077562", "699077563",  "487223604",  "1191885783",  "1191885780",  "1191885781" , "1191885778", "126730088", "491000664" , "29324581",  "1191885774" , "126730026"]
    my_sim.add_tracked_edges(tracked_edges)

    # Add scheduled events from a JSON file (can be dictionary). Use the format as in example_incident.json
    #my_sim.add_events("a20_exercise/a20_scenario/example_incident.json")

    # Add a new route that vehicles can be assigned to.
    my_sim.add_route(("urban_in_e", "urban_out_w"), "new_route")

    # These individual functions above can be replaced as below, where the 'parameters.json' file contains
    # a dictionary of all necessary parameters (under 'edges', 'junctions', 'phases', 'controllers' and 'events')
    # my_sim.load_objects("parameters.json")

    # This file can either be created manually, or by saving objects in previous simulations. This is done
    # using the save_objects function as below.
    #my_sim.save_objects("objects.json")

    # Add a function that is called on each new vehicle in the simulation. Simulation parameters are; curr_step,
    # vehicle_id, route_id, vehicle_type, departure, origin, destination. These values are filled automatically.
    # For other parameters, use a parameters dictionary as below. Use add_vehicle_out_funcs() for functions
    # called when vehicles exit the simulation (only with vehicle_id and/or curr_step). Vehicle in/out functions
    # can be removed using remove_vehicle_[in/out]_functions().

    # MR: wonder if I can use this to flexibly change the vehicle type on the fly? (yes)

    def stochastic_av_vehicle_class(simulation, vehicle_id, avShare=0):
        """
        Reassign vehicle ID to AV type if random roll < avShare
        """
        if simulation.vehicle_exists(vehicle_id):
            if randint(0,100)/100 < avShare:
                simulation.set_vehicle_vals(vehicle_id,type="AV")
                vehicleClass[vehicle_id]="AV"
            else:
                vehicleClass[vehicle_id]="HV"

    def set_BA_CACC_vehicle(simulation,vehicle_id,share=0):
        """
        Reassign vehicle ID to CACC_BA
        """
        if vehicleClass[vehicle_id] == "AV" and randint(0,100)/100 < share:
            simulation.set_vehicle_vals(vehicle_id,type="BA_AV")
        

    def set_BA_HV_vehicle(simulation,vehicle_id,share=0):
        """
        Reassign vehicle ID to HV_BA
        """
        if vehicleClass[vehicle_id] == "HV" and randint(0,100)/100 < share:
            simulation.set_vehicle_vals(vehicle_id,type="BA_cars")


    my_sim.add_vehicle_in_functions(stochastic_av_vehicle_class, parameters={"avShare": avShare})
    my_sim.add_vehicle_in_functions(set_BA_CACC_vehicle, parameters={"share": BA_CACC_share})
    my_sim.add_vehicle_in_functions(set_BA_HV_vehicle, parameters={"share": BA_HV_share})

    n, sim_dur, warmup = 1, 750 / my_sim.step_length, 0 / my_sim.step_length
    
    my_sim.set_tl_metering_rate(rm_id="crooswijk_meter", metering_rate=4000) #4000 should boil down to 'allow all'
    my_sim.set_tl_metering_rate(rm_id="a13_meter", metering_rate=4000)
    
    if warmup > 0:
        my_sim.step_through(n_steps=warmup, pbar_max_steps=sim_dur+warmup, keep_data=False)

    while my_sim.curr_step < sim_dur + warmup:

        
        
        # Step through n seconds.
        my_sim.step_through(n_seconds=n, pbar_max_steps=sim_dur+warmup)

        # if my_sim.curr_step == 100 / my_sim.step_length:
        #     my_sim.cause_incident(100, n_vehicles=2, edge_speed=5)

        # Check speed in merging area to decide if VSLs should be active or not (try to mimic what RWS does?)
        
        if my_sim.curr_step >= 50 / my_sim.step_length and (my_sim.curr_step % 50 / my_sim.step_length == 0) and useVSL:
            vsl1MergeSpeed = my_sim.get_interval_detector_data(detector_ids="vsl1SpeedRef", data_keys="speeds", n_steps=10, interval_end=0, avg_step_vals=True, avg_det_vals=True)
            if vsl1MergeSpeed < VSL_threshold_70_input.value and vsl1MergeSpeed > VSL_threshold_50_input.value:
                my_sim.controllers["vsl1"].set_speed_limit(70)
            if vsl1MergeSpeed < VSL_threshold_50_input.value:
                my_sim.controllers["vsl1"].set_speed_limit(50)
            if vsl1MergeSpeed > 100:
                my_sim.controllers["vsl1"].deactivate()

            # vsl2MergeSpeed = my_sim.get_interval_detector_data(detector_ids="a13_rm_downstream", data_keys="speeds", n_steps=10, interval_end=0, avg_step_vals=True, avg_det_vals=True)
            # if vsl2MergeSpeed < 80:
            #     my_sim.controllers["vsl2"].set_speed_limit(70)
            # if vsl2MergeSpeed < 60:
            #     my_sim.controllers["vsl2"].set_speed_limit(50)
            # if vsl2MergeSpeed > 100:
            #     my_sim.controllers["vsl2"].deactivate()

        # Set ramp metering rate.
        if useRM and (my_sim.curr_step % 50 / my_sim.step_length == 0): #Every 50 steps 
            #
            # You can input your controller logic here!
            #
            CWweavingSectionSpeed = my_sim.get_interval_detector_data(detector_ids="cw_rm_downstream", data_keys="speeds", n_steps=10, interval_end=0, avg_step_vals=True, avg_det_vals=True)
            if (CWweavingSectionSpeed < RM_threshold_input.value) :
                #Linearly metering more heavily as speed moves below threshold
                my_sim.set_tl_metering_rate(rm_id="crooswijk_meter", metering_rate=4000*(130-CWweavingSectionSpeed)/130)
            else:
                #Not metering at all
                my_sim.set_tl_metering_rate(rm_id="crooswijk_meter", metering_rate=4000)
            




    # End the simulation.
    my_sim.end()
    return vehicleClass  #hopefully this is by reference?

avShare = penRate_slider.value
if show_Control_toggle.value:
    vslState = VSL_toggle.value
    rmState = RM_toggle.value
else:
    vslState = False
    rmState = False

forceGUI = see_GUI_toggle.value
scenario_name="A20_CIEM6220_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)
scenario_savefile=savesDir+"A20_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)+".pkl"
scenario_vclass_savefile=savesDir+"A20_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)+"_vehTypeDict.pkl"
scenario_ssm_savefile=savesDir+"A20_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)+"_ssm.xml"
cache_savefile=cacheDir+"A20_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)+".pkl"
cache_vclass_safefile=cacheDir+"A20_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)+"_vehTypeDict.pkl"
cache_ssm_savefile=cacheDir+"A20_"+str(avShare)+"_"+str(vslState)+"_"+str(rmState)+"_ssm.xml"
BA_CACC_share=0
BA_HV_share=0

try:
    if BA_button.clicked:
        #Behavioral adaptation logic, prior to running another simulation
        BA_CACC_share=CACC_BA_penrate_slider.value
        BA_HV_share=HV_BA_penrate_slider.value
        #Build dicts based on all available inputs
        CACC_user_values_dict={'minGap':CACC_minGap_slider.value,'accel':CACC_accel_slider.value,'decel':CACC_decel_slider.value,'tau':CACC_tau_slider.value,'speedControlGainCACC':CACC_SCG.value,'gapClosingControlGainGap':CACC_GCGG.value,'gapClosingControlGainGapDot':CACC_GCGGD.value, 
                            'gapControlGainGap':CACC_GCG.value,'gapControlGainGapDot':CACC_GCGD.value,'collisionAvoidanceGainGap':CACC_CAG.value,'collisionAvoidanceGainGapDot':CACC_CAGD.value}
        HV_user_values_dict={'minGap':HV_minGap_slider.value,'accel':HV_accel_slider.value,'decel':HV_decel_slider.value,'sigma':HV_sigma_slider.value,'tau':HV_tau_slider.value}
        updateVehicleClasses(HV_user_values_dict=HV_user_values_dict,CACC_user_values_dict=CACC_user_values_dict)
except NameError as e:
    #If the button isn't shown, skip this part entirely
    pass
    

if run_sim_button.clicked:
    #Caching logic
    if use_cache_toggle.value: #boolean
        #Check if file exists, load that instead of running an actual simulation if that's the case
        print(f'[CACHE] Checking cache for simulation scenario {scenario_name}...')
        if os.path.exists(cache_savefile) and os.path.exists(cache_vclass_safefile) and os.path.exists(cache_ssm_savefile):
            my_sim = cache_savefile
            vClass=pd.read_pickle(cache_vclass_safefile)
            simComplete=True
            shutil.copy(cache_savefile,savesDir+'curr_sim.pkl')
            shutil.copy(cache_ssm_savefile,savesDir+'')
            with open(savesDir+'curr_sim.txt','w') as f: #Artificial description if simulation is cache-loaded
                f.write(f"Description: CIEM6220U1 Dutch A20 Motorway scenario. VSL: {vslState}, RM: {rmState}, AV Pen. Rate: {round(avShare*100,0)}. *")
            print(f'[CACHE] Scenario {scenario_name} found and loaded.')
        else:
            print(f'[CACHE] Scenario files {cache_savefile} or {cache_vclass_safefile} not found in cache.')

    if not simComplete:
        print(f'Simulating scenario...')
        # Initialise the simulation object.
        scenario_desc = f"CIEM6220U1 Dutch A20 Motorway scenario. VSL: {vslState}, RM: {rmState}, AV Pen. Rate: {round(avShare*100,0)}." #Procedurally generated scenario description for later readability
        my_sim = Simulation(scenario_name, scenario_desc=scenario_desc)
        # Add "-seed {x}" to the command line to set a seed for the simulation
        sim_seed = "1" if "-seed" not in sys.argv[:-1] else sys.argv[sys.argv.index("-seed")+1]
        seed(int(sim_seed))
        try:
            vClass=runScenario(my_sim,useVSL=vslState,useRM=rmState)
        except Exception as e:
            print("Scenario computation failed. Error detail:", e)
            pass #Simply discard this simulation
        
        simComplete=True
        if save_sim_toggle.value: #boolean
            #my_sim.save_data(savesDir+"A20_"+str(avShare)+"_"+str(vslState)+".pkl")
            my_sim.save_data(scenario_savefile)
            #my_sim.print_summary(save_file=savesDir+"A20_"+str(avShare)+"_"+str(vslState)+".txt") #summary not needed, not really
            #with open(savesDir+"A20_"+str(avShare)+"_"+str(vslState)+"_vehTypeDict.pkl",'wb') as h: #rather, save the vehicle classes in their own named pickle file
            with open(scenario_vclass_savefile,'wb') as h:
                pickle.dump(vClass,h)
                #and move the ssm measurements into ./experiments with the appropriate renaming as well
            shutil.copy("./a20_exercise/a20_scenario/a20ssm.xml",scenario_ssm_savefile)
        #if compare_sim_results_toggle.value:
        my_sim.save_data('./experiments/curr_sim.pkl') #just do it anyway, cleaner.
        my_sim.print_summary('./experiments/curr_sim.txt')
        shutil.copy("./a20_exercise/a20_scenario/a20ssm.xml",scenario_ssm_savefile)  #just do it anyway, also cleaner.
        
    

# Simulation results

In [ ]:
# Visualisation cells: show relevant sim results directly from my_sim object (skip saving)
if simComplete and not compare_sim_results_toggle.value:
    plt = Plotter(my_sim,time_unit="minutes")

elif simComplete and compare_sim_results_toggle.value:
    old_sim='./experiments/old_sim.pkl'
    my_sim_file='./experiments/curr_sim.pkl'
    #Print relevant simulation information here
    old_sim_desc='./experiments/old_sim.txt'
    my_sim_desc='./experiments/curr_sim.txt'
    print("Current simulation scenario:\n" + readSimDescription(my_sim_desc))
    print("Previous simulation scenario:\n" + readSimDescription(old_sim_desc))
    mplt = MultiPlotter(scenario_label=f"{readSimDescription(my_sim_desc)} [current] vs \n {readSimDescription(old_sim_desc)} [previous] \n",time_unit="minutes")
    mplt.add_simulations([my_sim_file, old_sim], labels=["Current scenario","Previous scenario"] )
    

## Weaving section space-time diagram

In [ ]:
trackedEdges=["126729958","126710337","1191885785"] #hardcoded for convenience, could allow for some selection with another input widget if I so please - but then needs a good label dictionary to avoid having these numerical IDs - and a good explanatory figure!

if simComplete and not compare_sim_results_toggle.value:
    plt.plot_space_time_diagram(trackedEdges)
elif simComplete and compare_sim_results_toggle.value:
    # This needs to be done with two plt objects loaded and plotted manually one by one (oof.)
    plt = Plotter(my_sim_file,time_unit="minutes")
    plt.plot_space_time_diagram(trackedEdges)

    plt2 = Plotter(old_sim,time_unit="minutes")
    plt2.plot_space_time_diagram(trackedEdges)


## Weaving section detector speed

In [ ]:
if simComplete and not compare_sim_results_toggle.value:
    plt.plot_detector_data("cw_rm_downstream", "speeds", aggregation_steps=10)
elif simComplete and compare_sim_results_toggle.value:
    try:
        mplt.plot_detector_data("cw_rm_downstream", "speeds", aggregation_steps=10)
    except ValueError:
        pass

## Control outputs

In [ ]:
#Only show any output if one or more control was active
if simComplete and not (vslState or rmState):
    print("No control outputs to showcase. Please re-run simulation with active control.")
if simComplete and not compare_sim_results_toggle.value and vslState:
    plt.plot_vsl_data("vsl1")
elif simComplete and compare_sim_results_toggle.value and vslState:
    try:
        plt.plot_vsl_data("vsl1")
        plt2.plot_vsl_data("vsl1")
    except Exception as e:
        pass
if simComplete and not compare_sim_results_toggle.value and rmState:
    try:
        plt.plot_rm_rate("crooswijk_meter")
        plt.plot_rm_queuing("crooswijk_meter")
    except Exception as e:
        pass
elif simComplete and compare_sim_results_toggle.value and rmState:
    try:
        plt.plot_rm_rate("crooswijk_meter")
        plt.plot_rm_queuing("crooswijk_meter")
        plt2.plot_rm_rate("crooswijk_meter")
        plt2.plot_rm_queuing("crooswijk_meter")
    except Exception as e:
        pass

## Network-wide Total Time Spent

In [ ]:
if simComplete and not compare_sim_results_toggle.value:
    plt.plot_vehicle_data(data_key="tts")
elif simComplete and compare_sim_results_toggle.value:
    try:
        mplt.plot_vehicle_data(data_key="tts")
    except ValueError:
        pass

## Surrogate Safety Measures (SSMs)

In [ ]:
#for now this only works for the latest simulation
if simComplete:
    try:
        df=pd.read_xml(scenario_ssm_savefile, xpath="/SSMLog/conflict").reset_index()
        df2=pd.read_xml(scenario_ssm_savefile, xpath="/SSMLog/conflict/TTCSpan").reset_index()
        df3=pd.merge(df,df2)
        df3['vehClass']=df3['ego'].apply(lambda row: vClass[row])
        df3['values_fl']=df3['values'].apply(lambda x: [float(v) for v in x.split(" ") if v != "NA"])
        df3 = df3.explode('values_fl')
        #For QoL: get scenario type from filename and set title accordingly
        df3.boxplot(column=["values_fl"],by="vehClass",showfliers=False).set_title("Time-to-Collision").get_figure().suptitle("")
    except Exception as e:
        print("TTC measure results not available. Please rerun simulation.")
        print(e)

In [ ]:
if simComplete:
    try:
        df4=pd.read_xml(scenario_ssm_savefile, xpath="/SSMLog/conflict/DRACSpan").reset_index()
        df5=pd.merge(df,df4)
        df5['vehClass']=df5['ego'].apply(lambda row: vClass[row])
        df5['values_fl']=df5['values'].apply(lambda x: [float(v) for v in x.split(" ") if v != "NA"])
        df5 = df5.explode('values_fl')
        #For QoL: get scenario type from filename and set title accordingly
        df5.boxplot(column=["values_fl"],by="vehClass",showfliers=False).set_title("Deceleration Rate to Avoid a Crash (DRAC)").get_figure().suptitle("")
    except Exception as e:
        print("DRAC measure results not available. Please rerun simulation.")
        print(e)



In [ ]:
if simComplete:
        try:
                df6=pd.read_xml(scenario_ssm_savefile, xpath="/SSMLog/conflict/PET").reset_index()
                df7=pd.merge(df,df6)
                df7['vehClass']=df7['ego'].apply(lambda row: vClass[row])
                #df7['values_fl']=df7['value'].apply(lambda x: [float(v) for v in x.split(" ") if v != "NA"])
                df7.boxplot(column=["value"],by="vehClass",showfliers=True).set_title("Post-Encroachment Time (PET)").get_figure().suptitle("")
                
        except Exception as e:
                print("PET measure results not available. Please rerun simulation.")
                print(e)      


In [ ]:
if simComplete:
        try:
            #df9=pd.read_xml(scenario_ssm_savefile, xpath="/SSMLog/globalMeasures").reset_index()
            df10=pd.read_xml(scenario_ssm_savefile, xpath="/SSMLog/globalMeasures/BRSpan").reset_index()
            #df10=pd.merge(df9,df8)
            #df10['vehClass']=df10['ego'].apply(lambda row: vClass[row])        
            df10['values_fl']=df10['values'].apply(lambda x: [float(v) for v in x.split(" ") if v != "NA"])
            df10.boxplot(showfliers=False).set_title("Braking Rate (BR)").get_figure().suptitle("") #This being a global measure there (should) be no need / option for isolating this on a per-vehicle basis
        except Exception as e:
                print("BR measure results not available. Please rerun simulation.")
                print(e)     

In [ ]:
#Finally, replace old with current simulation, where applicable
if simComplete:
    shutil.copy("./experiments/curr_sim.pkl","./experiments/old_sim.pkl")
    shutil.copy("./experiments/curr_sim.txt","./experiments/old_sim.txt")
